In [1]:
import os

import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from src.helper import prepare_dataframe
from src.main import read_data, coop_data, get_coordinates_for_prontos, find_min_dist_to_next_pronto, preprocess_finance_df, merge_dataframes, read_csv, create_csv

C:\Users\Fabio\Documents\6. Semester\dawr\Assignment2\src
C:\Users\Fabio\Documents\6. Semester\dawr\Assignment2\src


C:\Users\Fabio\Documents\6. Semester\dawr\Assignment2\src\util.py:50: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df.replace('---', pd.np.nan, inplace=True)
C:\Users\Fabio\Documents\6. Semester\dawr\Assignment2\src\util.py:50: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df.replace('---', pd.np.nan, inplace=True)
C:\Users\Fabio\Documents\6. Semester\dawr\Assignment2\src\util.py:68: FutureWarning: The default value of regex will change from True to False in a future version.
  df_coordinates['Ortschaftsname'] = df_coordinates['Ortschaftsname'].str.replace(' b. Willisau', '')
C:\Users\Fabio\Documents\6. Semester\dawr\Assignment2\src\util.py:50: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df.replace('---', pd.np.nan, i

# Helper Functions

These functions train a model on your data and use cross validation to determine the average negative mean squared error. You do not have to edit these! However, it might make sense to add new helper functions here.

In [2]:
def scoreLinear(X, y):
    """Trains a linear regression model and evaluates it with 3-fold cross validation.
  
    Parameters: 
    X (array): Training data of shape (n_samples, n_features)
    y (array): Target values of shape (n_samples,)
  
    Returns: 
    Float: average score of the model after cross validation
    """
    regr = linear_model.LinearRegression()
    scores = cross_val_score(regr, X, y, cv=3, scoring='neg_mean_squared_error')
    return scores.mean()

In [3]:
def scoreBoosting(X, y):
    """Trains a gradient boosting model and evaluates it with 3-fold cross validation.
  
    Parameters: 
    X (array): Training data of shape (n_samples, n_features)
    y (array): Target values of shape (n_samples,)
  
    Returns: 
    Float: average score of the model after cross validation
    """
    clf = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=2, random_state=0, loss='ls')
    scores = cross_val_score(clf, X, np.ravel(y), cv=3, scoring='neg_mean_squared_error')
    return scores.mean()

In [4]:
def dataframeToXy(df, predict_column, feature_columns):
    """Convert the dataframe to a format usable for the ML algorithms"""
    X = df[feature_columns].values.reshape(-1, df[feature_columns].shape[1]) # all features
    y = df[[predict_column]].values.reshape(-1, 1) # values to predict
    return X, y

In [5]:
def runScoring(df, predict_column, feature_columns):
    """This runs both algorithms to determine the scores for all given features in the dataset.
    
    Parameters: 
    df (dataframe): dataframe with Score and features
    predict_column: name of the value to predict, e.g. 'Score'
    feature_columns: list of all column names that are to be used as features
  
    Returns: 
    linear: negative mean squared error of linear regression
    boost: negative mean squared error of boosting algorithm
    """
    X, y = dataframeToXy(df, predict_column, feature_columns)
    boost = scoreBoosting(X, y)
    linear = scoreLinear(X, y)
    return linear, boost

In [6]:
def runScoringSimple(df, predict_column):
    """Alternative to function above, this takes all the columns in the 
    dataframe as features except the single column which is used for the score.
    """
    all_feature_columns = list(df.columns)
    all_feature_columns.remove(predict_column)
    return runScoring(df, predict_column, all_feature_columns)

In [7]:
def createBaselineRandom(df, predict_column):
    """creates random features and returns the average score after 20 rounds"""
    np.random.seed(0)
    baseline = df[['Score']].copy()
    averageLinearScore = 0
    averageBoostingScore = 0
    rounds = 20
    for i in range(rounds):
        baseline['feature1'] = np.random.randint(0, 100, df.shape[0])
        baseline['feature2'] = np.random.randint(0, 100, df.shape[0])
        baseline['feature3'] = np.random.randint(0, 100, df.shape[0])
        lin, boost = runScoring(baseline, predict_column, ['feature1', 'feature2', 'feature3'])
        averageLinearScore += lin / rounds
        averageBoostingScore += boost / rounds
    print(f'linear   : {averageLinearScore}')
    print(f'boosting : {averageBoostingScore}')

# Load Data

Recommended: Make sure you have a column named 'Score', preferably as the first column, while the name of the municipality is the index.


In [8]:
ranking = read_csv('data', 'safety-ranking.csv')
# benenne Gemeinde in Gemeindename um
ranking = ranking.rename(columns={'Gemeinde': 'Gemeindename'})
ranking = ranking.set_index('Gemeindename')
ranking

C:\Users\Fabio\Documents\6. Semester\dawr\Assignment2\src\main.py:126: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df.replace('---', pd.np.nan, inplace=True)


,Score,Total Anzahl Personen Sozialhilfe,Total Verunfallte Personen,Total Anzahl Unfälle 2021
Gemeindename,,,,
Doppleschwand,0.000000,0.0,0.0,0.0
Honau,0.089783,0.0,1.0,1.0
Romoos,0.161609,0.0,3.0,0.0
Flühli,0.215479,7.0,1.0,1.0
Vitznau,0.233435,9.0,0.0,2.0
...,...,...,...,...
Horw,10.055665,351.0,29.0,61.0
Ebikon,15.550368,443.0,69.0,108.0
Kriens,27.024600,1004.0,73.0,141.0


These are the features that were used to create the original ranking in Assignment 1:  
**TODO: Please write down your original features here!**


- Total Anzahl Personen Sozialhilfe
- Total Verunfallte Personen
- Total Anzahl Unfälle

# Get a first baseline for your model with random values

In [9]:
# This code creates a baseline for your model, using random features
createBaselineRandom(ranking, 'Score')

C:\Users\Fabio\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:286: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  warnings.warn(
C:\Users\Fabio\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:286: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  warnings.warn(
C:\Users\Fabio\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:286: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  warnings.warn(
C:\Users\Fabio\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:286: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  warnings.warn(
C:\Users\Fabio\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:286: FutureWarning: The loss 'ls' was deprecated in v1.0 and 

linear   : -174.09733300234578
boosting : -267.36629876029264


C:\Users\Fabio\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:286: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  warnings.warn(
C:\Users\Fabio\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:286: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  warnings.warn(
C:\Users\Fabio\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:286: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  warnings.warn(


**WARNING**: This uses the same dataframe for both the linear model and the boosting model. In your code you will want to use different ones for the different models. Your goal is to get both of these scores to as close as 0 as possible and most likely you will need to do different steps for linear regression and for gradient boosting and therefore have different features. 

# Get a second baseline by using all features you have

In [12]:
# this assumes we have a dataframe with a column named Score and all other columns are features.
# You might have to change this code if your dataframe looks different.

prepare_dataframe(ranking)
#df_final = read_csv('data', 'engineering.csv')
#df_final = df_final.set_index('Gemeindename')
#df_final['Score'] = ranking['Score']
#unset index
#df_final.reset_index(inplace=True)

#create_csv('data','engineering_final.csv',df_final)
df_final = read_csv('data', 'engineering_final.csv')
df_final = df_final.set_index('Gemeindename')
df_final = df_final.drop(columns=['Total Anzahl Dossiers'])
df_final = df_final.drop(columns=['Total Sozialhilfequote'])
df_final = df_final.drop(columns=['Total Sozialhilfequote Schweizer'])
df_final = df_final.drop(columns=['Total Sozialhilfequote Ausländer'])

linear, boost = runScoringSimple(df_final, 'Score')
print(f'linear   : {linear}')
print(f'boosting : {boost}')

df_final

C:\Users\Fabio\Documents\6. Semester\dawr\Assignment2\src\main.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  df_coordinates['Ortschaftsname'] = df_coordinates['Ortschaftsname'].str.replace(' b. Willisau', '')


linear   : -114.32643463820283
boosting : -75.3818386923129


C:\Users\Fabio\Documents\6. Semester\dawr\Assignment2\src\main.py:126: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df.replace('---', pd.np.nan, inplace=True)
C:\Users\Fabio\Documents\6. Semester\dawr\Assignment2\src\main.py:126: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df.replace('---', pd.np.nan, inplace=True)
C:\Users\Fabio\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:286: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  warnings.warn(
C:\Users\Fabio\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:286: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  warnings.warn(
C:\Users\Fabio\anaconda3\lib\site-packages\sklearn\ensemble\_gb.

,Score,Alter 0-17,Alter 18-64,Alter 65+,obligatorische Schule,sekundarstufe I,tertiärstufe,Personenschaden absolut mit Personenschaden,Personenschaden pro 1000 Einwohner,Durchschnitt Anzahl 2017-2021,...,Finanzvermögen¹,Verwaltungsvermögen,Total Passiven,Fremdkapital,Eigenkapital,Spezielfinanzierung EK,Fond im EK,Aufwertungsreserven,überiges Eigentkapital,Bilanzüberschuss
Gemeindename,,,,,,,,,,,,,,,,,,,,,
Adligenswil,1.203089,1.2,0.9,0.2,0.0,0.2,0.0,3,0.5,4.0,...,40621889,36196780,76818669,43315758,33502911,6979702,299028,0,0,26224182
Aesch,0.520740,0.5,1.2,0.0,0.0,0.0,0.0,1,0.8,3.0,...,11552799,9913351,21466150,7704460,13761690,8167324,0,2860,0,5591506
Alberswil,0.700305,2.6,1.3,0.0,0.0,0.0,0.0,4,6.0,3.0,...,4955889,3367123,8323012,4375329,3947683,1287875,30763,0,0,2629044
Altbüron,0.718262,6.1,2.0,0.0,0.0,0.0,0.0,1,1.0,2.0,...,6513672,5530778,12044450,8502752,3541699,1254746,38801,0,0,2248152
Altishofen,1.005567,2.1,1.3,0.0,0.0,0.0,0.0,5,2.5,4.0,...,13569690,13773478,27343168,6820858,20522311,4876009,56841,4775488,0,10813973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Honau,0.089783,0.0,0.0,0.0,0.0,0.0,0.0,1,2.4,1.0,...,5369088,45734,5414822,2694182,2720640,1765375,163816,0,0,791449
Rain,1.490393,0.0,0.0,0.0,0.0,0.0,0.0,12,4.1,5.0,...,36640455,14631698,51272154,35526066,15746088,4744277,28687,2159032,0,8814092
Romoos,0.161609,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,2402308,5181736,7584044,1241493,6342551,579826,204486,1365246,0,4192993


# Do your own feature engineering here

Your goal is to find the right features to get both scores as close to 0 as possible. Add more columns to your dataframe and choose which ones of those you will use for the machine learning models. Use the methods discussed in class to improve your results even further.

In [11]:
#TODO: Insert all your code here 